# Idea for notebook configs

Could be useful somewhere (not sure where) with a reusable NotebookConfig utility, so that notebooks just declares a plain defaults dict, overrides with vars from a .env file if present, and gets a ready-to-use config object with attributes. 

Here’s an example of how a utils/config.py could look:

In [ ]:
# utils/config.py
from decouple import Config, RepositoryEnv
from pathlib import Path

class NotebookConfig:
    """
    Simple notebook-friendly configuration loader.

    Usage:
        defaults = {
            "VAR1": "default_value",
            "VAR2": 123,
            "FLAG": False,
        }
        CONFIG = NotebookConfig(defaults)
        print(CONFIG.VAR1)
    """
    def __init__(self, defaults, env_file=None):
        """
        :param defaults: dict of {key: default_value}
        :param env_file: optional Path to .env file; defaults to notebook folder
        """
        if env_file is None:
            env_file = Path().absolute() / ".env"
        self._config = Config(RepositoryEnv(env_file))
        
        for key, default in defaults.items():
            cast_type = None
            if isinstance(default, bool):
                cast_type = bool
            elif isinstance(default, int):
                cast_type = int
            elif isinstance(default, float):
                cast_type = float

            value = self._config(key, default=default, cast=cast_type)
            setattr(self, key, value)

    def as_dict(self):
        """Return all config values as a dict."""
        return {k: getattr(self, k) for k in self.__dict__ if not k.startswith("_")}

    def pretty_print(self):
        """Print all config keys and values."""
        import json
        print(json.dumps(self.as_dict(), indent=2))

Usage in a notebook:

# notebook top
from utils.config import NotebookConfig
from datetime import date

# default values for this notebook
defaults = {
    # DHIS2 connection
    "DHIS2_BASE_URL": "https://play.im.dhis2.org/stable-2-42-3-1",
    "DHIS2_USERNAME": "admin",
    "DHIS2_PASSWORD": "district",
    "DHIS2_DRY_RUN": False,
    "DHIS2_TIMEZONE_OFFSET": 0,

    # ERA5 import
    "IMPORT_VARIABLE": "total_precipitation",
    "IMPORT_VALUE_COL": "tp",
    "IMPORT_FROM_UNITS": "m",
    "IMPORT_TO_UNITS": "mm",
    "IMPORT_START_DATE": "2025-01-01",
    "IMPORT_END_DATE": date.today().isoformat(),

    # Download
    "DOWNLOAD_FOLDER": "../../guides/data/local",
    "DOWNLOAD_PREFIX": "era5_hourly_sierra_leone",

    # Aggregation
    "TEMPORAL_AGGREGATION": "sum",
    "SPATIAL_AGGREGATION": "mean"
}

CONFIG = NotebookConfig(defaults)
CONFIG.pretty_print()  # optional, to see actual values in effect

# access values like attributes
print(CONFIG.DHIS2_BASE_URL)
print(CONFIG.IMPORT_TO_UNITS)